## Production Planning Project: **Phase 2**


### Details

Full Name: **Hamed Araab**

Student Number: **9925003**


### Dependencies

First, we import the libraries that we are going to need later on.

- `pandas` and `numpy` for minor data manipulations.
- `pulp` for creating and solving the LP problem.


In [33]:
import math

import numpy as np
import pandas as pd

from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, lpSum

### Dataset

Here, we import the forecasting data from the previous phase.


In [34]:
phase_1_output = pd.read_excel(
    "../phase_1/output.xlsx",
    sheet_name="Data",
    index_col=0,
)

phase_1_output

,G1 Actual,G2 Actual,G3 Actual,G1 Forecast (SES),G2 Forecast (SES),G3 Forecast (SES),G1 Forecast (SMA),G2 Forecast (SMA),G3 Forecast (SMA),G1 Forecast (WMA),G2 Forecast (WMA),G3 Forecast (WMA),G1 Forecast (LR),G2 Forecast (LR),G3 Forecast (LR),G1 Forecast (ALR),G2 Forecast (ALR),G3 Forecast (ALR)
0,352.1,146.8,14.7,352.100000,146.800000,14.700000,352.100000,146.800000,14.700000,352.10000,146.800000,14.700000,395.830000,182.258571,17.584286,334.085280,135.515172,14.582346
1,469.3,171.7,16.2,352.100000,146.800000,14.700000,469.300000,171.700000,16.200000,469.30000,171.700000,16.200000,395.457368,182.059248,17.071203,426.822969,160.227161,17.641170
2,498.0,229.3,22.7,387.260000,154.270000,15.150000,498.000000,229.300000,22.700000,498.00000,229.300000,22.700000,395.084737,181.859925,16.558120,508.436307,230.524012,21.458910
3,476.5,220.7,19.2,420.482000,176.779000,17.415000,439.800000,182.600000,17.866667,460.21000,195.520000,19.150000,394.712105,181.660602,16.045038,487.321661,222.172337,18.686954
4,434.7,206.6,17.7,437.287400,189.955300,17.950500,481.266667,207.233333,19.366667,481.51000,213.480000,19.650000,394.339474,181.461278,15.531955,454.033856,207.979807,17.713604
5,405.4,185.6,14.7,436.511180,194.948710,17.875350,469.733333,218.866667,19.866667,459.90000,215.370000,19.150000,393.966842,181.261955,15.018872,413.492525,186.840937,15.739125
6,388.2,176.7,10.2,427.177826,192.144097,16.922745,438.866667,204.300000,17.200000,428.41000,198.920000,16.500000,393.594211,181.062632,14.505789,395.386796,177.730740,13.005491
7,311.8,176.4,14.7,415.484478,187.510868,14.905921,409.433333,189.633333,14.200000,402.66000,185.350000,13.050000,393.221579,180.863308,13.992707,349.805020,175.707265,13.510873
8,301.4,168.1,12.2,384.379135,184.177608,14.844145,368.466667,179.566667,13.200000,353.44000,178.330000,13.350000,392.848947,180.663985,13.479624,301.400000,168.100000,12.200000
9,291.4,161.1,10.5,359.485394,179.354325,14.050902,333.800000,173.733333,12.366667,321.88000,172.310000,12.550000,392.476316,180.464662,12.966541,291.400000,161.100000,10.500000


### Assumptions

Here are the assumptions that are made:


In [35]:
# the optimal forecasting methods according to the results of the first phase
selector = [
    "G1 Forecast (WMA)",
    "G2 Forecast (ALR)",
    "G3 Forecast (ALR)",
]

T = range(20, 26)  # periods
G = range(1, 4)  # product groups

# demands
D = (
    phase_1_output[selector]
    .rename(columns=dict(zip(selector, G)))
    .transpose()
    .pipe(np.ceil)[T]
)

### Decision Variables

The decision variables of the problem are the following:


In [36]:
VarSet = LpVariable.dicts

RP = VarSet("RP", (T, G), lowBound=0, cat=LpInteger)  # regular production (units)
OP = VarSet("OP", (T, G), lowBound=0, cat=LpInteger)  # overtime production (units)
PI = VarSet("PI", (T, G), lowBound=0, cat=LpInteger)  # production increase (units)
PD = VarSet("PD", (T, G), lowBound=0, cat=LpInteger)  # production decrease (units)
IL = VarSet("IL", (T, G), cat=LpInteger)  # inventory level (units)
IS = VarSet("IS", (T, G), lowBound=0, cat=LpInteger)  # inventory surplus (units)
IG = VarSet("IG", (T, G), lowBound=0, cat=LpInteger)  # inventory shortage (units)
TW = VarSet("TW", T, lowBound=0, cat=LpInteger)  # total workers (worker)
OW = VarSet("OW", T, lowBound=0, cat=LpInteger)  # overtime workers (worker)
HW = VarSet("HW", T, lowBound=0, cat=LpInteger)  # hired workers (worker)
FW = VarSet("FW", T, lowBound=0, cat=LpInteger)  # fired workers (worker)

### LP Model

Here, we define the LP problem using `pulp`. To do so, we define a function that
returns a model with the given parameters. Thus, we can pass different
parameters to the function to get different models and conduct sensitivity
analysis with ease in the final section.


In [37]:
def get_model(
    rpr=3.125 * 7 / 9,
    opr=3.125 * 2 / 9,
    rpc=dict(zip(G, (11_300_000, 12_200_000, 16_700_000))),
    opc=dict(zip(G, (13_560_000, 14_640_000, 20_040_000))),
    pic=1_000_000,
    pdc=1_500_000,
    isc=dict(zip(G, (2_300_000 / 12, 3_100_000 / 12, 5_600_000 / 12))),
    igc=dict(zip(G, (0, 0, 0))),
    rs=15_000_000,
    os=4_500_000,
    hc=2_400_000,
    fc=12_000_000,
    iw=20_000,
    irp=dict(zip(G, (0, 0, 0))),
    iil=dict(zip(G, (0, 0, 0))),
    fil=dict(zip(G, (0, 0, 0))),
):
    """
    Parameters:
        `rpr`: regular production rate (units per worker).
        `opr`: overtime production rate (units per worker).
        `rpc`: regular production cost (tomans per unit).
        `opc`: overtime production cost (tomans per unit).
        `pic`: production increase cost (tomans per unit).
        `pdc`: production decrease cost (tomans per unit).
        `isc`: inventory surplus cost (tomans per unit).
        `igc`: inventory shortage cost (tomans per unit).
        `rs`: regular salary (tomans per worker).
        `os`: overtime salary (tomans per worker).
        `hc`: hiring cost (tomans per worker).
        `fc`: firing cost (tomans per worker).
        `iw`: initial workers (worker).
        `irp`: initial regular production (units).
        `iil`: initial inventory level (units).
        `fil`: final inventory level (units).
    Returns:
        A `pulp.LpProblem`.
    """

    model = LpProblem(name="production_planning", sense=LpMinimize)

    model += lpSum(
        [
            rpc[g] * RP[t][g]
            + opc[g] * OP[t][g]
            + pic * PI[t][g]
            + pdc * PD[t][g]
            + isc[g] * IS[t][g]
            + igc[g] * IG[t][g]
            for g in G
            for t in T
        ]
    ) + lpSum([rs * TW[t] + os * OW[t] + hc * HW[t] + fc * FW[t] for t in T])

    for t in T:
        model += lpSum([RP[t][g] for g in G]) <= rpr * TW[t]
        model += lpSum([OP[t][g] for g in G]) <= opr * TW[t]
        model += lpSum([OP[t][g] for g in G]) <= opr * OW[t]

        if t != T[0]:
            model += TW[t] == TW[t - 1] + HW[t] - FW[t]
        else:
            model += TW[t] == iw + HW[t] - FW[t]

        for g in G:
            model += IL[t][g] == IS[t][g] - IG[t][g]

            if t != T[0]:
                model += RP[t][g] == RP[t - 1][g] + PI[t][g] - PD[t][g]
                model += IL[t][g] == IL[t - 1][g] + RP[t][g] + OP[t][g] - D[t][g]
            else:
                model += RP[t][g] == irp[g] + PI[t][g] - PD[t][g]
                model += IL[t][g] == iil[g] + RP[t][g] + OP[t][g] - D[t][g]

            if t == T[-1]:
                model += IL[t][g] == fil[g]

    return model

We can access the model by calling `get_model`:


In [38]:
model = get_model()

Subsequently, we call `model.solve` and let PuLP solve it.


In [39]:
model.solve()

1

### Results

The results can be depicted by making use of `pd.DataFrame`.


In [40]:
print("total cost:", model.objective.value())

pd.DataFrame(
    {
        "RP": {f"T{t}": tuple(int(RP[t][g].value()) for g in G) for t in T},
        "OP": {f"T{t}": tuple(int(OP[t][g].value()) for g in G) for t in T},
        "PI": {f"T{t}": tuple(int(PI[t][g].value()) for g in G) for t in T},
        "PD": {f"T{t}": tuple(int(PD[t][g].value()) for g in G) for t in T},
        "IL": {f"T{t}": tuple(int(IL[t][g].value()) for g in G) for t in T},
        "IS": {f"T{t}": tuple(int(IS[t][g].value()) for g in G) for t in T},
        "IG": {f"T{t}": tuple(int(IG[t][g].value()) for g in G) for t in T},
        "TW": {f"T{t}": int(TW[t].value()) for t in T},
        "OW": {f"T{t}": int(OW[t].value()) for t in T},
        "HW": {f"T{t}": int(HW[t].value()) for t in T},
        "FW": {f"T{t}": int(FW[t].value()) for t in T},
    }
).transpose()

total cost: 300943835000.0


,T20,T21,T22,T23,T24,T25
RP,"(435, 152, 6)","(435, 152, 6)","(435, 152, 6)","(435, 152, 6)","(436, 154, 5)","(436, 154, 5)"
OP,"(0, 0, 0)","(1, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
PI,"(435, 152, 6)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(1, 2, 0)","(0, 0, 0)"
PD,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 1)","(0, 0, 0)"
IL,"(-5, -14, -1)","(-4, -21, -1)","(-2, -23, -1)","(-2, -19, 0)","(-1, 3, 0)","(0, 0, 0)"
IS,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 3, 0)","(0, 0, 0)"
IG,"(5, 14, 1)","(4, 21, 1)","(2, 23, 1)","(2, 19, 0)","(1, 0, 0)","(0, 0, 0)"
TW,244,244,244,244,245,245
OW,0,2,0,0,0,0
HW,0,0,0,0,1,0


As can be seen, salaries are much higher than firing cost and we have too many
workers with respect to the demands. Thus, we need to fire the majority of
workers and adopt a strategy similar to Level.


### Sensitivity Analysis

In the final section, we conduct sensitivity analysis on two parameters: `rs`
and `fc`.

As mentioned earlier, we can easily achieve this task by passing different
parameter values to `get_model` in a for loop.


#### `rs`: Regular Salary (Tomans per Worker)


In [41]:
for rs in [12e6, 13e6, 14e6, 15e6, 16e6, 17e6, 18e6]:
    model = get_model(rs=rs)

    print(f"rs = {int(rs)},\ttotal cost:", model.objective.value())

rs = 12000000,	total cost: 296545835000.0
rs = 13000000,	total cost: 298011835000.0
rs = 14000000,	total cost: 299477835000.0
rs = 15000000,	total cost: 300943835000.0
rs = 16000000,	total cost: 302409835000.0
rs = 17000000,	total cost: 303875835000.0
rs = 18000000,	total cost: 305341835000.0


#### `fc`: Firing Cost (Tomans per Worker)


In [42]:
for fc in [6e6, 8e6, 10e6, 12e6, 14e6, 16e6, 18e6]:
    model = get_model(fc=fc)

    print(f"fc = {int(fc)},\ttotal cost:", model.objective.value())

fc = 6000000,	total cost: 182407835000.0
fc = 8000000,	total cost: 221919835000.0
fc = 10000000,	total cost: 261431835000.0
fc = 12000000,	total cost: 300943835000.0
fc = 14000000,	total cost: 340455835000.0
fc = 16000000,	total cost: 379967835000.0
fc = 18000000,	total cost: 419479835000.0


### Exporting the Model

The model can be exported and saved as a JSON file using PuLP's API.


In [43]:
model.to_json("./model.json")